In [124]:
# Importar librerias
import pandas as pd
import numpy as np
import json
import requests
import os

# **1. Obtén los datos de la siguientes fuente de apis: https://analytics.deacero.com/api/expuesta/sqlexp/api_key**

Nota 1: Los api_keys válidos se proporcionan por correo. Los cuales son los siguientes:

**API / TOKEN**

* Lineas Aereas / fed214f3-332d-522c-97ac-da395a066dba

* Pasajeros 2016 / 3689da48-d557-5e5f-8347-006ced354939

* Pasajeros 2017 / 2a323bb8-0a6d-5bd5-8366-90041c4f1c8c

* Vuelos 2016 / 2743ebad-f1e2-5eff-8c4d-f8c5191d1775

* Vuelos 2017 / a6960833-d5a3-56dc-b125-da9e4e1fce69

Nota 2: los API cambiaron de lugar al siguiente enlace https://analytics.deacero.com/api/teenus/get-data/api_key

In [125]:
la=pd.read_json('https://analytics.deacero.com/api/teenus/get-data/fed214f3-332d-522c-97ac-da395a066dba')
p16=pd.read_json('https://analytics.deacero.com/api/teenus/get-data/3689da48-d557-5e5f-8347-006ced354939')
p17=pd.read_json('https://analytics.deacero.com/api/teenus/get-data/2a323bb8-0a6d-5bd5-8366-90041c4f1c8c')
v16=pd.read_json('https://analytics.deacero.com/api/teenus/get-data/2743ebad-f1e2-5eff-8c4d-f8c5191d1775')
v17=pd.read_json('https://analytics.deacero.com/api/teenus/get-data/a6960833-d5a3-56dc-b125-da9e4e1fce69')

In [126]:
print(la.shape)
print(p16.shape)
print(p17.shape)
print(v16.shape)
print(v17.shape)

(5, 2)
(100, 3)
(100, 3)
(200, 6)
(200, 6)


# **2. Se tiene un requerimiento de análisis de las fuentes de datos de pasajeros y viajes. Como podrás observar, las listas se han estado llevando por año. En esta primera parte del reto se te pide:**

* Unir cada conjunto de datos en una sola lista.


* Explicar el proceso realizado.

Para unir las bases de los PASAJEROS y VUELOS del año 2016 y 2017 se uso la funcion concat donde los datos fueron agrupados siguiendo el mismo formato

* En caso de detectar anomalías generadas por esta unión, deberás indicar el tipo de anomalía que se presenta, como se puede resolver y resolverlo de ser posible.

Entre las anomalias que se pudieran tener fue que en la lista de pasajeros del 2017 se repetian los registros de pasajeros por lo que solo se opto por eliminar los datos duplicados


In [127]:
#Unir las bases de los PASAJEROS del año 2016 y 2017 usando la funcion concat
pALL=pd.concat([p16, p17], ignore_index=True)

In [128]:
#Entre las anomalias que se tuvo fue que en la lista de pasajeros del 2017 se repetian los registros de pasajeros
#por lo que se opto por eliminar los datos duplicados

sta = pALL.pivot_table(columns=['ID_Pasajero','Pasajero'], aggfunc='size')
sta

#Tambien se puede usar un boolean para verificar si hay valores repetidos/duplicados. 
#boolean = p16['ID_Pasajero'].duplicated().any()
#print(boolean) #Si hay duplicados imprime True, si no hay imprime False

ID_Pasajero  Pasajero           
38           Anemone Ratner         10
191          Dan Reichenbach         2
402          Julia Barnett           1
550          Christopher Bridges     2
553          Nancy Lomonaco          1
                                    ..
729          Kendra Daniel           2
730          Timothy Moore           3
741          Jennifer Neal           1
742          Jessica Lewis           2
             Lori Lopez              1
Length: 138, dtype: int64

In [129]:
pALL=pALL.drop_duplicates()
print(pALL.shape)

(138, 3)


In [130]:
pALL.sort_values(by=['ID_Pasajero'])

,ID_Pasajero,Pasajero,Edad
119,38,Anemone Ratner,39
149,191,Dan Reichenbach,55
176,402,Julia Barnett,33
129,550,Christopher Bridges,54
91,553,Nancy Lomonaco,34
...,...,...,...
150,729,Kendra Daniel,59
103,730,Timothy Moore,21
117,741,Jennifer Neal,31
153,742,Jessica Lewis,43


In [131]:
#Unir las bases de los VUELOS del año 2016 y 2017 nuevamente mediante concat
vALL=pd.concat([v16, v17], ignore_index=True)

In [132]:
sta2 = vALL.pivot_table(columns=['Viaje','Cve_Cliente'], aggfunc='size')
sta2

Viaje      Cve_Cliente
1/1/2016   642            1
1/1/2017   682            5
1/10/2017  582            1
1/11/2016  595            1
           668            1
                         ..
9/6/2016   576            1
9/7/2016   557            1
           574            1
9/7/2017   619            1
9/9/2016   575            1
Length: 377, dtype: int64

In [133]:
vALL.sort_values(by=['Viaje','Cve_Cliente'])

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente
35,KL,1/1/2016,First Class,204,DAL-TUL,642
206,UA,1/1/2017,Economy,78,DAL-HOU,682
210,UA,1/1/2017,Economy,90,DAL-PHX,682
214,SW,1/1/2017,Economy,72,DAL-AMA,682
216,AA,1/1/2017,Business,115,DAL-LGA,682
...,...,...,...,...,...,...
166,AA,9/6/2016,Business,186,DAL-AMA,576
109,AA,9/7/2016,Economy,196,DAL-HOU,557
160,AA,9/7/2016,First Class,108,DAL-AUS,574
347,AV,9/7/2017,First Class,163,DAL-SLC,619


# **3. De lo obtenido anteriormente se requiere relacionar las listas de vuelos y pasajeros, que permitan analizar el perfil del pasajero por cada vuelo efectuado. De tal forma que se puedan obtener datos consolidados. En esta parte deberás:**

* Explicar el proceso que fue utilizado para unir los pasajeros y los vuelos.

Se tienen dos DF de Pasajeros y Vuelos de distintos tamaños, las cuales se quieren unir y que a la vez comparten las columnas 'ID_Pasajero' y 'Cve_Cliente', respectivamente.
Se opto por hacer el cambio del nombre de la columna 'Cve_Cliente' por 'ID_Pasajero' y usar la funcion merge.

* Qué tipo de relación y por qué.

Se uso la funcion merge para complementar el DF de los vuelos con la información de los pasajeros mediante la columna 'ID_Pasajero' que ahora los dos DF comparten. 
Se uso esta relacion porque el DF de vuelos habia repeticiones en la columna de ID_Pasajero.

In [134]:
#Se tienen dos DF de Pasajeros y Vuelos de distintos tamaños, las cuales se quieren unir y que a la vez comparten 
#las columnas 'ID_Pasajero' y 'Cve_Cliente', respectivamente.

#Se opto por hacer el cambio del nombre de la columna 'Cve_Cliente' por 'ID_Pasajero' para poder usar la funcion merge 
#y complementar el DF de los vuelos con la información de los pasajeros

In [135]:
print(pALL.shape)
print(vALL.shape)

(138, 3)
(400, 6)


In [136]:
vALL.rename(columns = {'Cve_Cliente':'ID_Pasajero'}, inplace = True)
ALLDATA = pd.merge(vALL, pALL, on='ID_Pasajero')


In [137]:
ALLDATA

,Cve_LA,Viaje,Clase,Precio,Ruta,ID_Pasajero,Pasajero,Edad
0,SW,9/10/2016,Economy,60,DAL-MDW,553,Nancy Lomonaco,34
1,SW,11/18/2016,Business,81,DAL-PHX,553,Nancy Lomonaco,34
2,AA,6/11/2016,Economy,150,DAL-SLC,554,Naresj Patel,25
3,DA,9/21/2016,Economy,68,DAL-AMA,554,Naresj Patel,25
4,AM,7/13/2016,First Class,233,DAL-HOU,554,Naresj Patel,25
...,...,...,...,...,...,...,...,...
429,AV,11/15/2017,Economy,150,DAL-OKC,619,Timothy Adkins,66
430,AV,9/1/2017,First Class,166,DAL-LGA,729,Kendra Daniel,59
431,AA,6/27/2017,Economy,119,DAL-AMA,729,Kendra Daniel,59
432,AM,5/24/2017,Business,194,DAL-AMA,625,Monique Ramirez,35


In [138]:
#ALLDATA = pd.merge(vALL, pALL, left_on='Cve_Cliente', right_on='ID_Pasajero')

In [139]:
#ALLDATA

# **4. Ahora se requiere que los datos consolidados de los vuelos y pasajeros se puedan unir con los datos de las Líneas Aéreas. En el caso de que la línea aérea no se pueda relacionar con la de vuelos y pasajeros se deberá indicar que se trata de “Otra” y finalmente se deberá dejar únicamente las columnas:**

* Fecha del viaje
* Clase
* Precio
* Ruta
* Edad
* Línea Aérea

En esta parte deberás indicar:

* ¿Qué tipo de proceso consideraste para unir los datos que se piden?

Se considera un proceso similar al del punto 3 mediante la función merge

* ¿Qué columnas utilizaste para lograr esa relación?

Para lograr la relacion se usaron las columnas equivalentes de 'Cve_LA' y 'Code' ya que finalmente se quiere tener el nombre de la aerolinea descrita en el DF con la información de los pasajeros y vuelos.

* ¿Qué tipo de unión utilizaste para unir los datos?

El tipo de conjuncion que fue utilizado dentro de la función merge fue mediante 'outer' ya que precisamente se queria complementar la tabla usando la union (es decir, informacion que este presente en ambas tablas) en base a dos columnas de las distintos DF

* ¿Qué tipo de proceso utilizaste para dejar únicamente las columnas que se piden?

Se definio una nueva DF considerando solo las columnas requeridas.

In [140]:
sta3= ALLDATA.pivot_table(columns=['Cve_LA'], aggfunc='size')
sta3

Cve_LA
AA    76
AM    88
AV    87
DA     7
KL    73
SW    95
UA     8
dtype: int64

In [141]:
la

,Code,Linea_Aerea
0,AA,American Airlines
1,SW,Southwest
2,AM,Aeromexico
3,AV,Avianca
4,KL,KLM


In [142]:
ALLDATA.columns

Index(['Cve_LA', 'Viaje', 'Clase', 'Precio', 'Ruta', 'ID_Pasajero', 'Pasajero',
       'Edad'],
      dtype='object')

In [143]:
la.columns

Index(['Code', 'Linea_Aerea'], dtype='object')

In [162]:
MERGEDDATA = pd.merge(ALLDATA, la, how='outer', left_on='Cve_LA', right_on='Code')

In [163]:
MERGEDDATA

,Cve_LA,Viaje,Clase,Precio,Ruta,ID_Pasajero,Pasajero,Edad,Code,Linea_Aerea
0,SW,9/10/2016,Economy,60,DAL-MDW,553,Nancy Lomonaco,34,SW,Southwest
1,SW,11/18/2016,Business,81,DAL-PHX,553,Nancy Lomonaco,34,SW,Southwest
2,SW,6/25/2016,Economy,65,DAL-OKC,557,Natalie DeCherney,63,SW,Southwest
3,SW,12/2/2016,Economy,126,DAL-MDW,557,Natalie DeCherney,63,SW,Southwest
4,SW,6/25/2016,Economy,66,DAL-AMA,557,Natalie DeCherney,63,SW,Southwest
...,...,...,...,...,...,...,...,...,...,...
429,KL,6/17/2017,Business,217,DAL-SLC,717,Billy Whitney,37,KL,KLM
430,KL,1/29/2017,Economy,131,DAL-HOU,669,Jason Crane,32,KL,KLM
431,KL,9/21/2017,Business,266,DAL-AUS,714,Sheena Morgan,56,KL,KLM
432,KL,7/16/2017,First Class,223,DAL-AMA,671,David Lee,62,KL,KLM


In [146]:
MERGEDDATA.fillna('Other', inplace=True)

In [147]:
MERGEDDATA

,Cve_LA,Viaje,Clase,Precio,Ruta,ID_Pasajero,Pasajero,Edad,Code,Linea_Aerea
0,SW,9/10/2016,Economy,60,DAL-MDW,553,Nancy Lomonaco,34,SW,Southwest
1,SW,11/18/2016,Business,81,DAL-PHX,553,Nancy Lomonaco,34,SW,Southwest
2,SW,6/25/2016,Economy,65,DAL-OKC,557,Natalie DeCherney,63,SW,Southwest
3,SW,12/2/2016,Economy,126,DAL-MDW,557,Natalie DeCherney,63,SW,Southwest
4,SW,6/25/2016,Economy,66,DAL-AMA,557,Natalie DeCherney,63,SW,Southwest
...,...,...,...,...,...,...,...,...,...,...
429,KL,6/17/2017,Business,217,DAL-SLC,717,Billy Whitney,37,KL,KLM
430,KL,1/29/2017,Economy,131,DAL-HOU,669,Jason Crane,32,KL,KLM
431,KL,9/21/2017,Business,266,DAL-AUS,714,Sheena Morgan,56,KL,KLM
432,KL,7/16/2017,First Class,223,DAL-AMA,671,David Lee,62,KL,KLM


In [148]:
MERGEDDATA.columns

Index(['Cve_LA', 'Viaje', 'Clase', 'Precio', 'Ruta', 'ID_Pasajero', 'Pasajero',
       'Edad', 'Code', 'Linea_Aerea'],
      dtype='object')

In [149]:
MERGEDDATA.shape

(434, 10)

In [150]:
#Fecha del viaje
#Clase
#Precio
#Ruta
#Edad
#Línea Aérea
NEWMERGEDDATA= MERGEDDATA[['Viaje','Clase','Precio','Ruta','Edad','Linea_Aerea']].copy()

In [151]:
NEWMERGEDDATA.shape

(434, 6)

# **5. Por último, se requiere el promedio semestral (el primer semestre es de Ene - Jun y el segundo es de Jul - Dic) del precio agrupado por Año, Clase, Ruta y las Línea Aérea como columnas.**



In [152]:
#Pasar la columna Viaje al formato pd.to_time
NEWMERGEDDATA['Viaje'] = pd.to_datetime(NEWMERGEDDATA['Viaje'])

In [153]:
#Agregar la columna con el año
NEWMERGEDDATA['Year'] = NEWMERGEDDATA['Viaje'].dt.year

In [ ]:
NEWMERGEDDATA

In [155]:
#Separar los datos por semestre 
s1_2016 = NEWMERGEDDATA.loc[NEWMERGEDDATA["Viaje"].between("2016-01-01", "2016-06-30")]
s2_2016 = NEWMERGEDDATA.loc[NEWMERGEDDATA["Viaje"].between("2016-07-01", "2016-12-31")]
s1_2017 = NEWMERGEDDATA.loc[NEWMERGEDDATA["Viaje"].between("2017-01-01", "2017-06-30")]
s2_2017 = NEWMERGEDDATA.loc[NEWMERGEDDATA["Viaje"].between("2017-07-01", "2017-12-31")]


In [156]:
s1_2016.head()

,Viaje,Clase,Precio,Ruta,Edad,Linea_Aerea,Year
2,2016-06-25,Economy,65,DAL-OKC,63,Southwest,2016
4,2016-06-25,Economy,66,DAL-AMA,63,Southwest,2016
8,2016-05-09,Economy,95,DAL-AUS,21,Southwest,2016
13,2016-06-23,Business,97,DAL-LGA,20,Southwest,2016
14,2016-02-14,Business,265,DAL-TUL,27,Southwest,2016


In [157]:
#Promedio si queremos todos los datos juntos
#NEWMERGEDDATA.groupby(['Year','Clase','Ruta','Linea_Aerea'], as_index=False)['Precio'].mean()
#NEWMERGEDDATA.groupby(['Year','Clase','Ruta','Linea_Aerea']).mean()

In [158]:
proms1_2016=s1_2016.groupby(['Year','Clase','Ruta','Linea_Aerea']).mean()
proms1_2016

Precio  Edad
Year Clase       Ruta    Linea_Aerea                    
2016 Business    DAL-AMA American Airlines    58.0  64.0
                         Avianca             132.0  40.0
                         KLM                 184.0  64.0
                 DAL-ATL Aeromexico          218.0  22.0
                         American Airlines   102.0  36.0
...                                            ...   ...
     First Class DAL-SLC KLM                 214.0  52.0
                         Southwest            70.0  52.0
                 DAL-TUL Aeromexico          104.0  66.0
                         KLM                 204.0  68.0
                         Southwest           103.0  55.0

[97 rows x 2 columns]

In [159]:
proms2_2016=s2_2016.groupby(['Year','Clase','Ruta','Linea_Aerea']).mean()
proms2_2016

Precio  Edad
Year Clase       Ruta    Linea_Aerea                    
2016 Business    DAL-AMA American Airlines   186.0  60.0
                 DAL-ATL American Airlines   135.0  50.0
                         Avianca             107.0  60.0
                         KLM                 227.0  30.0
                         Southwest           185.0  32.0
...                                            ...   ...
     First Class DAL-SLC Aeromexico          185.0  44.5
                         American Airlines   239.0  66.0
                 DAL-TUL Aeromexico          214.0  68.0
                         Avianca             245.0  66.0
                         KLM                 186.0  65.0

[76 rows x 2 columns]

In [160]:
proms1_2017=s1_2017.groupby(['Year','Clase','Ruta','Linea_Aerea']).mean()
proms1_2017

Precio       Edad
Year Clase       Ruta    Linea_Aerea                             
2017 Business    DAL-AMA Aeromexico         146.000000  39.000000
                         Southwest          163.000000  46.000000
                 DAL-ATL KLM                167.000000  29.000000
                         Southwest          131.500000  49.500000
                 DAL-AUS American Airlines  110.666667  49.666667
...                                                ...        ...
     First Class DAL-SLC Avianca            137.000000  29.000000
                         Southwest          256.000000  53.000000
                 DAL-TUL Aeromexico          55.000000  67.000000
                         Avianca             78.000000  32.000000
                         KLM                242.500000  50.500000

[80 rows x 2 columns]

In [161]:
proms2_2017=s2_2017.groupby(['Year','Clase','Ruta','Linea_Aerea']).mean()
proms2_2017

Precio  Edad
Year Clase       Ruta    Linea_Aerea              
2017 Business    DAL-AMA Aeromexico    204.0  52.0
                         Avianca       206.0  39.5
                         KLM           121.0  46.0
                 DAL-ATL Aeromexico    201.0  32.0
                 DAL-AUS Aeromexico    122.0  53.0
...                                      ...   ...
     First Class DAL-PHX Avianca       247.0  49.0
                 DAL-SLC Avianca       163.0  66.0
                         Southwest     170.5  41.5
                 DAL-TUL Aeromexico    208.5  55.5
                         Avianca       189.0  48.0

[68 rows x 2 columns]

In [166]:
print(NEWMERGEDDATA["Precio"].mean())
print(s1_2016["Precio"].mean())
print(s2_2016["Precio"].mean())
print(s1_2017["Precio"].mean())
print(s2_2017["Precio"].mean())

156.2258064516129
164.88073394495413
147.73626373626374
150.072
161.71559633027522
